Preprocessing of the data
From the data extraction via Wikipedia to clean the dataframe (The main portion will start soon below)

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html_content = requests.get(url).text
soup = BeautifulSoup(html_content, "lxml")

extracted = soup.find('div', class_='mw-parser-output')
tabledata = extracted.table.tbody

column_names = ['Postalcode','Borough','Neighborhood']
TableTrimed = pd.DataFrame(columns = column_names)

for tri in tabledata.find_all('tr'):
   cnt=0
   for tdi in tri.find_all('td'):
       cnt=cnt+1
       if cnt ==1:
           Postem=tdi.text.strip('\n').replace(']','')
       elif cnt==2:
           Bortem=tdi.text.strip('\n').replace(']','')
           if Bortem.find('Not assigned') != -1:
               Bortem=np.nan
       elif cnt==3:
           Neitem=tdi.text.strip('\n').replace(']','')
           if Neitem.find('Not assigned') != -1:
               Neitem=Bortem
           TableTrimed = TableTrimed.append({'Postalcode': Postem,'Borough': Bortem,'Neighborhood': Neitem},ignore_index=True)

TableTrimedr=TableTrimed.dropna() 

Import geocoder library and create a definition that returns latitude and longitude by inputting a postal code

In [2]:
import geocoder
def returnll(Postal):
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(Postal))
    lati_long_coords = g.latlng
    return lati_long_coords

From the dataframe that has been cleaned, extract the postal code information.
Then, by using the definition described above, all the combinations of latitude and longitude for the postal codes are collected.
Because the data is stored as "List", convert the type to "Dataframe" and merge the latitude and longitude dataframe to the dataframe with postalcode, borough, and neiborhood.

In [3]:
PC=TableTrimedr['Postalcode']
lllist=[returnll(PC) for PC in PC.tolist()]
lldf = pd.DataFrame(lllist, columns=['Latitude', 'Longitude'])
TableTrimedr['Latitude'] = lldf['Latitude']
TableTrimedr['Longitude'] = lldf['Longitude']

<ipython-input-3-b652fac5e386>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TableTrimedr['Latitude'] = lldf['Latitude']
<ipython-input-3-b652fac5e386>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TableTrimedr['Longitude'] = lldf['Longitude']


In [4]:
TableTrimedrX=TableTrimedr.dropna() 
TableTrimedrX.head(12)

,Postalcode,Borough,Neighborhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.65512,-79.36264
3,M4A,North York,Victoria Village,43.72327,-79.45042
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.66253,-79.39188
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.66263,-79.52831
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.81139,-79.19662
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.70718,-79.31192
9,M1B,Scarborough,"Malvern, Rouge",43.65739,-79.37804
11,M3B,North York,Don Mills,43.65034,-79.55362
12,M4B,East York,"Parkview Hill, Woodbine Gardens",43.78574,-79.15875
13,M5B,Downtown Toronto,"Garden District, Ryerson",43.72168,-79.34352


# Below is the main portion for this part of the assignment

In [5]:
import folium
import requests 
import json 
import matplotlib.cm as cm
import matplotlib.colors as colors
import pandas as pd

from pandas.io.json import json_normalize 
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim 

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [6]:
address = 'Toronto, Ontario Canada'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto Canada are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto Canada are 43.6534817, -79.3839347.


create map of Tronto using latitude and longitude values then add markers to map

In [9]:
map_Tronto = folium.Map(location=[latitude, longitude], zoom_start=11)
for lat, lng, label in zip(TableTrimedrX['Latitude'], TableTrimedrX['Longitude'], TableTrimedrX['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Tronto)  
    
map_Tronto

Identify Foursquare credentials

In [10]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180604' # Foursquare API version
LIMIT = 500 # A default Foursquare API limit value

'Regent Park, Harbourfront'

Select the forth Neighborhood named "Regent Park, Harbourfront" and show the latitude and longitude

In [12]:
TableTrimedrX.loc[4, 'Neighborhood']

neighborhood_latitude = TableTrimedrX.loc[4, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = TableTrimedrX.loc[4, 'Longitude'] # neighborhood longitude value
neighborhood_name = TableTrimedrX.loc[4, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.66253000000006, -79.39187999999996.


Extract posts within 500m radius

In [16]:
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

results = requests.get(url).json()
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Show the top 5 nearby venues

In [18]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-18-f4c77c7fdb5e>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Queen's Park,Park,43.663946,-79.392180
1,Hart House Theatre,Theater,43.663571,-79.394616
2,Mercatto,Italian Restaurant,43.660391,-79.387664
3,Starbucks,Coffee Shop,43.659456,-79.390411
4,Nando's,Portuguese Restaurant,43.661728,-79.386391


In [19]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

18 venues were returned by Foursquare.


Define a function that returns near-by venues with input of a combination of latitudes and longitudes.

In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Return all venues included in Totonto data

In [21]:
Tronto_venues = getNearbyVenues(names=TableTrimedrX['Neighborhood'],
                                   latitudes=TableTrimedrX['Latitude'],
                                   longitudes=TableTrimedrX['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [22]:
print(Tronto_venues.shape)
Tronto_venues.head()

(1535, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.65512,-79.36264,Roselle Desserts,43.653447,-79.362017,Bakery
1,Parkwoods,43.65512,-79.36264,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Parkwoods,43.65512,-79.36264,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
3,Parkwoods,43.65512,-79.36264,The Yoga Lounge,43.655515,-79.364955,Yoga Studio
4,Parkwoods,43.65512,-79.36264,Body Blitz Spa East,43.654735,-79.359874,Spa


Count the numbers of neighborhood

In [23]:
Tronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Wilson Heights, Downsview North",18,18,18,18,18,18
Bayview Village,7,7,7,7,7,7
"Bedford Park, Lawrence Manor East",3,3,3,3,3,3
Berczy Park,17,17,17,17,17,17
"Birch Cliff, Cliffside West",11,11,11,11,11,11
"Brockton, Parkdale Village, Exhibition Place",2,2,2,2,2,2
Caledonia-Fairbanks,5,5,5,5,5,5
Cedarbrae,1,1,1,1,1,1
Central Bay Street,9,9,9,9,9,9


In [24]:
print('There are {} uniques categories.'.format(len(Tronto_venues['Venue Category'].unique())))

There are 227 uniques categories.


In [25]:
# one hot encoding
Tronto_onehot = pd.get_dummies(Tronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Tronto_onehot['Neighborhood'] = Tronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Tronto_onehot.columns[-1]] + list(Tronto_onehot.columns[:-1])
Tronto_onehot = Tronto_onehot[fixed_columns]

In [26]:
Tronto_onehot.shape

(1535, 227)

In [27]:
Tronto_grouped = Tronto_onehot.groupby('Neighborhood').mean().reset_index()
Tronto_grouped

,Neighborhood,Zoo Exhibit,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bike Trail,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Stop,Business Service,Butcher,Café,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hardware Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Lake,Latin American Restaurant,Leather Goods Store,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Residential Building (Apartment / Condo),Restaurant,Rock Climbing Spot,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Soccer Field,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Storage Facility,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Bathurst Manor, Wilson Heights, Downsview North",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000,0.0000,0.000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.055556,0.055556

In [28]:
Tronto_grouped.shape

(63, 227)

Show top 5 venues in each neighborhood

In [29]:
num_top_venues = 5

for hood in Tronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Tronto_grouped[Tronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor, Wilson Heights, Downsview North----
               venue  freq
0               Park  0.11
1  Fish & Chips Shop  0.06
2        Coffee Shop  0.06
3       Liquor Store  0.06
4         Steakhouse  0.06


----Bayview Village----
                 venue  freq
0            Gift Shop  0.14
1             Pharmacy  0.14
2                 Park  0.14
3        Grocery Store  0.14
4  Fried Chicken Joint  0.14


----Bedford Park, Lawrence Manor East----
              venue  freq
0          Pharmacy  0.33
1      Intersection  0.33
2  Sushi Restaurant  0.33
3       Zoo Exhibit  0.00
4      Noodle House  0.00


----Berczy Park----
           venue  freq
0  Grocery Store  0.12
1           Park  0.12
2           Café  0.06
3            Bar  0.06
4           Bank  0.06


----Birch Cliff, Cliffside West----
                  venue  freq
0  Fast Food Restaurant  0.18
1                   Gym  0.09
2                  Bank  0.09
3    Chinese Restaurant  0.09
4  Other Great Outdoors  0.09


--

4              Theater  0.06


----Richmond, Adelaide, King----
                venue  freq
0                Park   0.5
1              Bakery   0.5
2         Zoo Exhibit   0.0
3              Office   0.0
4  Miscellaneous Shop   0.0


----Roselawn----
                       venue  freq
0            Auto Dealership  0.12
1                        Gym  0.12
2  Middle Eastern Restaurant  0.12
3                Gas Station  0.12
4                 Restaurant  0.12


----Rouge Hill, Port Union, Highland Creek----
                        venue  freq
0  Construction & Landscaping  0.33
1        Gym / Fitness Center  0.33
2                        Park  0.33
3                 Zoo Exhibit  0.00
4                      Office  0.00


----Scarborough Village----
                venue  freq
0  Italian Restaurant  0.06
1               Diner  0.06
2             Brewery  0.06
3         Pizza Place  0.06
4    Sushi Restaurant  0.04


----St. James Town----
                        venue  freq
0  Construction

In [31]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Show the top 10 venues that has already been transposed by a fuction defined previously

In [43]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Tronto_grouped['Neighborhood']

for ind in np.arange(Tronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Tronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Wilson Heights, Downsview North",Park,Gym,Italian Restaurant,Pizza Place,Pub,Coffee Shop,Restaurant,Movie Theater,Sandwich Place,Burrito Place
1,Bayview Village,Diner,Fried Chicken Joint,Pizza Place,Grocery Store,Pharmacy,Park,Gift Shop,Elementary School,Dumpling Restaurant,Eastern European Restaurant
2,"Bedford Park, Lawrence Manor East",Pharmacy,Sushi Restaurant,Intersection,Yoga Studio,Donut Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
3,Berczy Park,Grocery Store,Park,Café,Athletics & Sports,Brazilian Restaurant,Bar,Bank,Bakery,Middle Eastern Restaurant,Pharmacy
4,"Birch Cliff, Cliffside West",Fast Food Restaurant,Pizza Place,Sandwich Place,Bank,Coffee Shop,Other Great Outdoors,Chinese Restaurant,Grocery Store,Gym,Pharmacy


Setup a cluster analysis condition

In [44]:
# set number of clusters
kclusters = 5
Tronto_grouped_clustering = Tronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Tronto_grouped_clustering)

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Tronto_merged = TableTrimedrX

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
Tronto_merged = Tronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Tronto_merged.head() # check the last columns!

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M3A,North York,Parkwoods,43.65512,-79.36264,2.0,Coffee Shop,Breakfast Spot,Yoga Studio,Theater,Italian Restaurant,Food Truck,Event Space,Electronics Store,Distribution Center,Pub
3,M4A,North York,Victoria Village,43.72327,-79.45042,2.0,Clothing Store,Furniture / Home Store,Men's Store,Coffee Shop,American Restaurant,Bookstore,Cosmetics Shop,Women's Store,Food Court,Toy / Game Store
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.66253,-79.39188,2.0,Coffee Shop,Sandwich Place,Gastropub,Mediterranean Restaurant,Falafel Restaurant,Café,Fried Chicken Joint,Bank,Italian Restaurant,Theater
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.66263,-79.52831,2.0,Pharmacy,Bank,Grocery Store,Park,Skating Rink,Shopping Mall,Café,Donut Shop,Falafel Restaurant,Event Space
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.81139,-79.19662,0.0,Zoo Exhibit,Fast Food Restaurant,Furniture / Home Store,Flea Market,Fish & Chips Shop,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room


Visualize the clusters on the map in Tronto

In [45]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

In [46]:
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Tronto_merged['Latitude'], Tronto_merged['Longitude'], Tronto_merged['Neighborhood'],kmeans.labels_):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker([lat, lon], radius=5, popup=label, color=rainbow[cluster-1], fill=True, fill_color=rainbow[cluster-1], fill_opacity=0.7).add_to(map_clusters)
map_clusters